In [28]:
import hail as hl
import numpy as np

In [ ]:
phen = '50_sim_inf'
n_chunks = 300
batch = '1'

In [ ]:
# mt = hl.read_matrix_table('gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_50_sim_inf_grouped300_batch_1_v2.mt/')
mt = hl.read_matrix_table('gs://nbaya/split/meta_split/ukb31063.hm3_'+phen+'_gmt'+str(n_chunks)+'_batch_'+batch+'.mt')

In [ ]:
sim_mt = hl.read_matrix_table('gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_'+phen+'_grouped'+str(n_chunks)+'_batch_'+batch+'_v3.mt')

In [ ]:
mt.describe()

In [ ]:
import seaborn as sns

In [ ]:
sns.kdeplot(mt.beta.take(mt.count_rows()))

In [ ]:
mt.beta.take(mt.count_rows())

In [ ]:
mt.write('gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_50_sim_inf_grouped300_batch_1_v3.mt',overwrite=True)

In [ ]:
mt.aggregate_cols(hl.agg.mean(mt.nonsim_phen))

In [ ]:
variant_set = 'qc_pos'

In [ ]:
mt = hl.read_matrix_table('gs://nbaya/split/ukb31063.'+variant_set+'_variants.gwas_samples_prerepart.mt')


In [ ]:
print(mt.describe())
print(mt.count())

In [ ]:
print(mt.count_rows())
mt = mt.sample_rows(0.01)
print(mt.count_rows())
if variant_set == 'hm3':
    mt = mt.repartition(1000)        
elif variant_set == 'qc_pos':
    mt = mt.repartition(5000)

# Get sample of UKB

In [1]:
phen='50_raw'
batch='1'
n_chunks=300
variant_set = 'hm3'

In [2]:
mt0 = hl.read_matrix_table('gs://nbaya/split/ukb31063.'+variant_set+'_variants.gwas_samples_repart.mt')

if phen == '50_raw':
    phen_tb_all = hl.import_table('gs://nbaya/ukb31063.50_raw.tsv.bgz',missing='',impute=True,types={'s': hl.tstr}).rename({phen: 'phen'})
else:
    phen_tb_all = hl.import_table('gs://phenotype_31063/ukb31063.phesant_phenotypes.both_sexes.tsv.bgz',
                                  missing='',impute=True,types={'"userId"': hl.tstr}).rename({ '"userId"': 's', '"'+phen+'"': 'phen'})

#Remove withdrawn samples
withdrawn = hl.import_table('gs://nbaya/w31063_20181016.csv',missing='',no_header=True)
withdrawn_set = set(withdrawn.f0.take(withdrawn.count()))
phen_tb_all.describe()
phen_tb_all = phen_tb_all.filter(hl.literal(withdrawn_set).contains(phen_tb_all['s']),keep=False) 

phen_tb = phen_tb_all.select(phen_tb_all['s'], phen_tb_all['phen'])
phen_tb = phen_tb.key_by('s')
phen_tb.count() #expect to be 361194 - 50 = 361144 for every phenotype before filtering, minus withdrawn samples 

mt1 = mt0.annotate_cols(phen_str = hl.str(phen_tb[mt0.s]['phen']).replace('\"',''))
mt1 = mt1.filter_cols(mt1.phen_str == '',keep=False)

if phen_tb.phen.dtype == hl.dtype('bool'):
    mt1 = mt1.annotate_cols(phen = hl.bool(mt1.phen_str)).drop('phen_str')
else:
    mt1 = mt1.annotate_cols(phen = hl.float64(mt1.phen_str)).drop('phen_str')


n_samples = mt1.count_cols()
print('\n>>> N samples = '+str(n_samples)+' <<<') #expect n samples to match n_non_missing from phenotypes.both_sexes.tsv
mt2 = mt1.add_col_index()

NameError: name 'hl' is not defined

In [10]:
group_size = int(n_samples/n_chunks)+1     #the ideal number of samples in each group
#list of group ids to be paired to each sample (Note: length of group_ids > # of cols in mt, but it doesn't affect the result)
group_ids = np.ndarray.tolist(np.ndarray.flatten(np.asarray([range(n_chunks)]*group_size))) 
group_ids = group_ids[0:n_samples]
randstate = np.random.RandomState(int(batch)) #seed with batch number
randstate.shuffle(group_ids)
mt3 = mt2.annotate_cols(group_id = hl.literal(group_ids)[hl.int32(mt2.col_idx)]) #assign group ids


In [14]:
mt4 = mt3.sample_rows(0.01)

In [18]:
mt5 = mt4.filter_cols(mt4.group_id < 5)

In [20]:
mt5.write('gs://nbaya/split/ukb31063.'+variant_set+'_variants.gwas_samples_'+phen+'.batch_'+batch+'.downsample_v2.mt') #Takes ~30 min with 50 workers

2018-12-13 13:07:26 Hail: INFO: Coerced sorted dataset
2018-12-13 13:13:00 Hail: INFO: wrote 11042 items in 1000 partitions to gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_50.batch_1.downsample.mt


In [19]:
mt5.count()

2018-12-13 13:05:03 Hail: INFO: Coerced sorted dataset
2018-12-13 13:05:45 Hail: INFO: Coerced sorted dataset


(11042, 6010)

In [21]:
print('gs://nbaya/split/ukb31063.'+variant_set+'_variants.gwas_samples_'+phen+'.batch_'+batch+'.downsample.mt')

gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_50.batch_1.downsample.mt


# Run conventional GWAS to obtain residuals

In [29]:
phen='50_raw'
batch='1'
variant_set = 'hm3'

In [7]:
mt0 = hl.read_matrix_table('gs://nbaya/split/ukb31063.'+variant_set+'_variants.gwas_samples_repart.mt')

if phen == '50_raw':
    phen_tb_all = hl.import_table('gs://nbaya/ukb31063.50_raw.tsv.bgz',missing='',impute=True,types={'s': hl.tstr}).rename({phen: 'phen'})
else:
    phen_tb_all = hl.import_table('gs://phenotype_31063/ukb31063.phesant_phenotypes.both_sexes.tsv.bgz',
                                  missing='',impute=True,types={'"userId"': hl.tstr}).rename({ '"userId"': 's', '"'+phen+'"': 'phen'})

#Remove withdrawn samples
withdrawn = hl.import_table('gs://nbaya/w31063_20181016.csv',missing='',no_header=True)
withdrawn_set = set(withdrawn.f0.take(withdrawn.count()))
phen_tb_all.describe()
phen_tb_all = phen_tb_all.filter(hl.literal(withdrawn_set).contains(phen_tb_all['s']),keep=False) 

phen_tb = phen_tb_all.select(phen_tb_all['s'], phen_tb_all['phen'])
phen_tb = phen_tb.key_by('s')
phen_tb.count() #expect to be 361194 - 50 = 361144 for every phenotype before filtering, minus withdrawn samples 

mt1 = mt0.annotate_cols(phen_str = hl.str(phen_tb[mt0.s]['phen']).replace('\"',''))
mt1 = mt1.filter_cols(mt1.phen_str == '',keep=False)

if phen_tb.phen.dtype == hl.dtype('bool'):
    mt1 = mt1.annotate_cols(phen = hl.bool(mt1.phen_str)).drop('phen_str')
else:
    mt1 = mt1.annotate_cols(phen = hl.float64(mt1.phen_str)).drop('phen_str')


n_samples = mt1.count_cols()
print('\n>>> N samples = '+str(n_samples)+' <<<') #expect n samples to match n_non_missing from phenotypes.both_sexes.tsv
mt2 = mt1.add_col_index()


Initializing Spark and Hail with default parameters...
Running on Apache Spark version 2.2.1
SparkUI available at http://10.128.0.11:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2-e60bdb1a125a
LOGGING: writing to /home/hail/hail-20181214-2046-0.2-e60bdb1a125a.log
2018-12-14 20:47:04 Hail: INFO: Reading table to impute column types
2018-12-14 20:47:07 Hail: INFO: Finished type imputation
  Loading column 's' as type 'str' (user-specified)
  Loading column '50_raw' as type 'float64' (imputed)
  Loading column 'phen_str' as type 'float64' (imputed)
2018-12-14 20:47:07 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (type not specified)



----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    's': str 
    'phen': float64 
    'phen_str': float64 
----------------------------------------
Key: []
----------------------------------------


2018-12-14 20:47:24 Hail: INFO: Coerced sorted dataset
2018-12-14 20:47:37 Hail: INFO: Coerced sorted dataset



>>> N samples = 360338 <<<


In [12]:
mt = mt1.rename({'dosage': 'x', 'phen': 'y'})
    
ht = mt.cols()
    
cov_list = [ ht['isFemale'], ht['age'], ht['age_squared'], ht['age_isFemale'],
            ht['age_squared_isFemale'] ]+ [ht['PC{:}'.format(i)] for i in range(1, 21)] 


linreg_results = ht.aggregate(hl.agg.linreg(y=ht.y, x = [1] + cov_list))
ht = ht.annotate(linreg = linreg_results)

2018-12-14 20:51:38 Hail: INFO: Coerced sorted dataset


In [19]:
linreg_results.beta

[179.59869007196897,
 -10.00499903973678,
 0.038712916919730854,
 -0.0017666884720448197,
 -0.11712029809520526,
 0.0010221997731952761,
 -42.78726612761782,
 9.424978891676345,
 -43.46895320117373,
 5.801306902992858,
 6.904967719772118,
 -13.915209419018991,
 20.632463911082937,
 8.403526326706062,
 -17.477925981211122,
 -19.25762979048966,
 -9.517242479473383,
 -13.81377183530335,
 -11.955458535270052,
 -0.9854647801955304,
 5.800489937988013,
 -21.47658234700417,
 -5.996207772991951,
 1.1453617677615708,
 15.555781939872313,
 -7.876648247655016]

In [21]:
ht = ht.annotate(y_pred =   linreg_results.beta[0]                +linreg_results.beta[1]*ht.isFemale    +linreg_results.beta[2]*ht.age+
                            linreg_results.beta[3] *ht.age_squared+linreg_results.beta[4]*ht.age_isFemale+linreg_results.beta[5]*ht.age_squared_isFemale+
                            linreg_results.beta[6] *ht.PC1        +linreg_results.beta[7]*ht.PC2         +linreg_results.beta[8]*ht.PC3+
                            linreg_results.beta[9] *ht.PC4        +linreg_results.beta[10]*ht.PC5        +linreg_results.beta[11]*ht.PC6+
                            linreg_results.beta[12]*ht.PC7        +linreg_results.beta[13]*ht.PC8        +linreg_results.beta[14]*ht.PC9+
                            linreg_results.beta[15]*ht.PC10       +linreg_results.beta[16]*ht.PC11       +linreg_results.beta[17]*ht.PC12+
                            linreg_results.beta[18]*ht.PC13       +linreg_results.beta[19]*ht.PC14       +linreg_results.beta[20]*ht.PC15+
                            linreg_results.beta[21]*ht.PC16       +linreg_results.beta[22]*ht.PC17       +linreg_results.beta[23]*ht.PC18+
                            linreg_results.beta[24]*ht.PC19       +linreg_results.beta[25]*ht.PC20)



In [23]:
ht = ht.annotate(res = ht.y-ht.y_pred)

In [27]:
ht.write('gs://nbaya/split/'+phen+'_linreg.ht')

2018-12-14 21:15:59 Hail: INFO: Coerced sorted dataset
2018-12-14 21:16:11 Hail: INFO: wrote 360338 items in 4 partitions


In [29]:
ht = mt_linreg.cols()

In [ ]:
ht.write('gs://nbaya/')

In [42]:
ht.annotate(y_pred = ht.linreg.beta[1]*ht.isFemale).y_pred.show()

FatalError: SparkException: Job 30 cancelled 

Java stack trace:
org.apache.spark.SparkException: Job 30 cancelled 
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1517)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1457)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:1704)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:1687)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:1676)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:630)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2029)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2126)
	at org.apache.spark.rdd.RDD$$anonfun$reduce$1.apply(RDD.scala:1026)
	at org.apache.spark.rdd

# Add residues to phen_tb

In [30]:
phen = '50_raw_res'
ht = hl.read_table('gs://nbaya/split/50_raw_linreg.ht')
phen_tb_all = hl.import_table('gs://nbaya/ukb31063.50_raw.tsv.bgz',missing='',impute=True,types={'s': hl.tstr})

2018-12-14 21:27:03 Hail: INFO: Reading table to impute column types
2018-12-14 21:27:04 Hail: INFO: Finished type imputation
  Loading column 's' as type 'str' (user-specified)
  Loading column '50_raw' as type 'float64' (imputed)
  Loading column 'phen_str' as type 'float64' (imputed)


In [31]:
phen_tb_all.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    's': str 
    '50_raw': float64 
    'phen_str': float64 
----------------------------------------
Key: []
----------------------------------------


In [35]:
hl.read_table('gs://nbaya/split/50_raw_linreg.ht').rename({'res': 'phen'}).describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    's': str 
    'isFemale': bool 
    'age': int32 
    'age_squared': int32 
    'age_isFemale': int32 
    'age_squared_isFemale': int32 
    'PC1': float64 
    'PC2': float64 
    'PC3': float64 
    'PC4': float64 
    'PC5': float64 
    'PC6': float64 
    'PC7': float64 
    'PC8': float64 
    'PC9': float64 
    'PC10': float64 
    'PC11': float64 
    'PC12': float64 
    'PC13': float64 
    'PC14': float64 
    'PC15': float64 
    'PC16': float64 
    'PC17': float64 
    'PC18': float64 
    'PC19': float64 
    'PC20': float64 
    'y': float64 
    'linreg': struct {
        beta: array<float64>, 
        standard_error: array<float64>, 
        t_stat: array<float64>, 
        p_value: array<float64>, 
        multiple_standard_error: float64, 
        multiple_r_squared: float64, 
        adjusted_r_squared: float64, 
        f_stat: float64, 
    